Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Data Loader

In [ ]:
import pathlib
from torch.nn.modules import loss
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from PIL import Image
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import sys
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


if torch.cuda.is_available():
    print("GPU available " + torch.cuda.torch.cuda.get_device_name())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

class GaitImage():
    def __init__(self, path, subject):           
        self.image = Image.open(path)
        self.subject = int(subject)

class GaitImageDataset(Dataset):
    def __init__(self, images):
        self.images = images

    def __len__(self):
        return len(self.images)

    def getSubjects(self):
        unique_subjects = set()

        for img in self.images:
            unique_subjects.add(img.subject)

        return unique_subjects


    def __getitem__(self, index):
        gait_image = self.images[index]
        image_PIL = gait_image.image

        transform_to_tensor = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Grayscale(),
            transforms.ToTensor()
        ])

        image = transform_to_tensor(image_PIL).to(device)

        label = torch.tensor(gait_image.subject)

        return image, label


def read_args():
    arguments = sys.argv
    dataset_arg = arguments[1]

    if dataset_arg not in ["oumvlp", "casiab74"]:
        raise ValueError(dataset_arg + " not supported, only `oumvlp` and `casiab74` allowed")

    model_arg = arguments[2]

    if model_arg not in ["vgg16", "resnet18", "resnet50"]:
        raise ValueError(model_arg + " not supported, only `vgg16`, `resnet18` and `resnet50` allowed")

    learning_rate = float(arguments[3])

    if learning_rate < 0:
        raise ValueError("Learning rate must be positive")

    max_epochs = int(arguments[4])

    return dataset_arg, model_arg, learning_rate, max_epochs

def load_datasets(dataset, batch_size=32, shuffle=True):
    train_folder = Path("/content/" + dataset +"/ProcessedBySID/train").resolve()
    query_folder = Path("/content/" + dataset +"/ProcessedBySID/query").resolve()

    train_images = []
    test_images = []

    for img in sorted(train_folder.iterdir()):
        train_images.append(GaitImage(img.as_posix(), img.name[:-8]))

    train_dataset = GaitImageDataset(train_images)

    for img in sorted(query_folder.iterdir()):
        test_images.append(GaitImage(img.as_posix(), img.name[:-8]))

    test_dataset = GaitImageDataset(test_images)

    train_loader = DataLoader(
        train_dataset,
        batch_size,
        shuffle
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size,
        shuffle
    )

    print("BATCH SIZE:    ", batch_size)
    
    return train_dataset, test_dataset, train_loader, test_loader

def get_model(model_arg, num_classes):
    # Load model
    model = torch.hub.load('pytorch/vision:v0.10.0', model_arg)

    if model_arg == "vgg16":
        # Modify conv layer to work with grayscale images
        model.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)

        # Get the number of input features to the original output layer
        in_features = model.classifier[6].in_features

        # Replace the last layer with a new fully connected layer
        model.classifier[6] = nn.Linear(in_features, num_classes)
    else:
        # Modify conv layer to work with grayscale images
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)

        # Get the number of input features to the original output layer
        in_features = model.fc.in_features

        # Replace the last layer with a new fully connected layer
        model.fc = nn.Linear(in_features, num_classes)

    model.to(device)

    return model

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total * 100
    return accuracy

def calculate_topk_accuracy(outputs, labels, k=1):
    _, predicted = torch.topk(outputs, k, dim=1)
    correct = torch.sum(predicted == labels.view(-1, 1))
    accuracy = correct.item() / labels.size(0) * 100
    return accuracy

def train_one_epoch(model, optimizer, loss_fn, train_loader):
    running_loss = 0.0
    avg_acc = 0.0
    avg_top5 = 0.0

    correct_labels = []
    predictions = []

    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # Zero your gradients for every batch
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        last_loss = loss.detach().item()
        running_loss += last_loss

        # Calculate accuracy
        top1_accuracy = calculate_topk_accuracy(outputs, labels, k=1)
        top5_accuracy = calculate_topk_accuracy(outputs, labels, k=5)
        avg_acc += top1_accuracy
        avg_top5 += top5_accuracy

        # Confusion matrix
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        correct_labels.extend(labels.cpu().numpy())

        if i % 1000 == 0:
            print('     batch {} loss: {} top-1 accuracy: {:.2f}% top-5 accuracy: {:.2f}%'.format(i, last_loss, top1_accuracy, top5_accuracy))


    cm = confusion_matrix(correct_labels, predictions)
    plt.figure(figsize=(18, 18))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

    return running_loss / len(train_loader), avg_acc / len(train_loader), avg_top5 / len(train_loader)

def test_one_epoch(model, loss_fn, test_loader):
    running_loss = 0.0
    avg_acc = 0.0
    avg_top5 = 0.0

    for i, data in enumerate(test_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Make predictions for this batch
        outputs = model(inputs)
       
        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)

        # Gather data and report
        last_loss = loss.detach().item()
        running_loss += last_loss

        # Calculate accuracy
        top1_accuracy = calculate_topk_accuracy(outputs, labels, k=1)
        top5_accuracy = calculate_topk_accuracy(outputs, labels, k=5)
        avg_acc += top1_accuracy
        avg_top5 += top5_accuracy
            
        if i % 1000 == 0:
            print('     batch {} loss: {} top-1 accuracy: {:.2f}% top-5 accuracy: {:.2f}%'.format(i, last_loss, top1_accuracy, top5_accuracy))

    return running_loss / len(test_loader), avg_acc / len(test_loader), avg_top5 / len(test_loader)

def main():
    dataset, model_arg, lr, max_epochs = read_args()

    print("---------------------INFO--------------------")
    print("DATASET:       ", dataset)
    print("MODEL:         ", model_arg)
    print("EPOCHS:        ", max_epochs)
    print("LEARNING RATE: ", lr)

    train_dataset, test_dataset, train_loader, test_loader = load_datasets(dataset, 32, True)

    print("---------------------TRAIN--------------------")
    print("Dataset size:        ", len(train_dataset))
    print("Unique classse size: ", len(train_dataset.getSubjects()))
    print("---------------------TEST---------------------")
    print("Dataset size:        ", len(test_dataset))
    print("Unique classse size: ", len(test_dataset.getSubjects()))
    print("----------------------------------------------")

    model = get_model(model_arg, len(train_dataset.getSubjects()))

    # Loss function
    loss_fn = nn.CrossEntropyLoss()
    loss_fn.to(device)

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr, weight_decay=0.001)

    for epoch in range(max_epochs):
        print("-------------------EPOCH {}------------------".format(epoch + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train()
        avg_loss, accuracy, top5_accuracy = train_one_epoch(model, optimizer, loss_fn, train_loader)

        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            # Add any other information you want to save
        }

        if epoch % 20 == 0:
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                # Add any other information you want to save
            }
            # Specify the path where you want to save the checkpoint
            checkpoint_path = model_arg + "_" + dataset + "-" + str(epoch)

            # Save the checkpoint to the specified path
            torch.save(checkpoint, checkpoint_path)

        # Set the model to evaluation mode, disabling dropout and using population
        # statistics for batch normalization.
        model.eval()

        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            avg_vloss, vaccuracy, vaccuracy_top5 = test_one_epoch(model, loss_fn, test_loader)

        print("------------------TRAIN------------------")
        print('Loss {} Accuracy: {:.2f}% Top-5 accuracy: {:.2f}%'.format(avg_loss, accuracy, top5_accuracy))
        print("------------------TEST-------------------")
        print('Loss {} Accuracy: {:.2f}% Top-5 accuracy: {:.2f}%'.format(avg_vloss, vaccuracy, vaccuracy_top5))

if __name__ == "__main__":
    main()

# CASIA-B

## Extract

In [ ]:
!unzip /content/drive/MyDrive/casiab.zip -d /content

## Process

In [ ]:
from logging import currentframe
import shutil
import os
from pathlib import Path


sourceFolder = Path("casiab74").resolve()
destinationFolder = "/ProcessedBySID/"
totalFiles = 0
currentFile = 0

destPath = sourceFolder.as_posix() + destinationFolder
Path.mkdir(Path(destPath), exist_ok = True)

def ExtractWalkingStatus(name):
    # Normal
    if  "nm" in name: return "nm"
    # In a coat
    if  "cl" in name: return "cl"
    # WIth a bag
    if  "bg" in name: return "bg"
    print("Error - Unknown walkign status: " + name)

def ProcessFile(src: Path, sid, ws, va, sn):
    #print("Processing {}/{}: ".format(currentFile, totalFiles) + src.name + " -> " + subjectID + "_" + walkingStatus + "_" + viewAnge + "_" + sequenceNumber, flush=True)
    shutil.copy(src, destPath + "_".join([sid, ws, va, sn]) + ".png")

def ProcessFile(category, src: Path, sid, i):
    #print("Processing {}/{}: ".format(currentFile, totalFiles) + src.name + " -> " + subjectID + "_" + str(i).zfill(3), flush=True)
    shutil.copy(src, destPath + "/" + category + "/"  + "_".join([str(sid).zfill(6), str(i).zfill(3)]) + ".png")

class DataImg(object):
    def __init__(self, sid, ws, va, sn, cat):
        self.subjectID = sid
        self.walkingStatus = ws
        self.viewAngle = va
        self.sequenceNumber = sn
        self.category = cat


"""
CASIA-B format:

CASIA-B Folder
    gallery
        subjectID1
            nm000_000001.png
            nm180_000002.png
            ...

        subjectID2
            bg072_000001.png
            bg090_000002.png
            ...

        ...
    query
        ...
    train
        ...

Image name:
WalkingStatusViewAngle_SequenceNumber.png

Subject ID - 4 numbers
Walking status - nm (normal), cl (in a coat), bg (with a bag)
View angle - 3 numbers (in degrees 0 - 180)
Sequence number - 2 numbers
"""

# Current folder contains Query, Train, Gallery

# Count files
totalFiles = 0
for i in sourceFolder.rglob("*.png"):
    totalFiles += 1

for category in sourceFolder.iterdir():
    if not category.name in ["gallery", "query", "train"]: continue
    Path.mkdir(Path(destPath + "/" + category.name), exist_ok = True)

    subjectID = 0
    for subject in category.iterdir():
        #subjectID = subject.name.removeprefix("00")
        subjectCounter = 0
        for image in subject.iterdir():
            split = image.name.split("_")
            walkingStatus = ExtractWalkingStatus(split[0])
            viewAngle = split[0].removeprefix(walkingStatus)
            sequenceNumber = split[1].removesuffix(".png").removeprefix("000")

            ProcessFile(category.name, image, subjectID, subjectCounter)
            subjectCounter += 1
            #ProcessFile(image, subjectID, walkingStatus, viewAngle, sequenceNumber)
            currentFile +=1
        if subjectCounter > 0:
            subjectID += 1

## Train
data_loader arguments: `dataset` `model` `learning_rate` `epochs`
- Datasets: `oumvlp`, `casiab74`
- Models: `vgg16`, `resnet18`, `resnet50`
- Learning rate: positive float
- Epochs: positive int

### VGGNet 16

In [ ]:
!python data_loader.py "casiab74" "vgg16" 0.00001 40

### ResNet 18

In [ ]:
!python data_loader.py "casiab74" "resnet18" 0.00001 40

### ResNet 50

In [ ]:
!python data_loader.py "casiab74" "resnet50" 0.0001 40

# OUMVLP

## Extract

In [ ]:
!unzip /content/drive/MyDrive/oumvlp.zip -d /content

## Process

In [ ]:
from logging import currentframe
import shutil
import os
from pathlib import Path


sourceFolder = Path("oumvlp").resolve()
destinationFolder = "/ProcessedBySID/"
totalFiles = 0
currentFile = 0

destPath = sourceFolder.as_posix() + destinationFolder
Path.mkdir(Path(destPath), exist_ok = True)


def ProcessFile(src: Path, sid, ws, va, sn):
    #print("Processing {}/{}: ".format(currentFile, totalFiles) + src.name + " -> " + subjectID + "_" + walkingStatus + "_" + viewAnge + "_" + sequenceNumber, flush=True)
    shutil.copy(src, destPath + "_".join([sid, ws, va, sn]) + ".png")

def ProcessFile(category, src: Path, sid, i):
    print("Processing {} {}/{}: ".format(category, currentFile, totalFiles) + src.name + " -> " + "_".join([str(sid).zfill(6), str(i).zfill(3)]), flush=True)
    shutil.copy(src, destPath + "/" + category + "/" + "_".join([str(sid).zfill(6), str(i).zfill(3)]) + ".png")

class DataImg(object):
    def __init__(self, sid, ws, va, sn, cat):
        self.subjectID = sid
        self.walkingStatus = ws
        self.viewAngle = va
        self.sequenceNumber = sn
        self.category = cat


"""
OUMVLP format:

OUMVLP Folder
    gallery
        subjectID1
            000_000001.png
            180_000002.png
            ...

        subjectID2
            210_000001.png
            270_000002.png
            ...

        ...
    query
        ...
    train
        ...

Image name:
ViewAngle_SequenceNumber.png

Subject ID - 4 numbers
View angle - 3 numbers (in degrees 0 - 180)
"""

# Current folder contains Query, Train, Gallery

# Count files
totalFiles = 0
for i in sourceFolder.rglob("*.png"):
    totalFiles += 1

for category in sourceFolder.iterdir():
    if not category.name in ["gallery", "query", "train"]: continue
    Path.mkdir(Path(destPath + "/" + category.name), exist_ok = True)

    subjectID = 0
    for subject in category.iterdir():
        subjectCounter = 0
        for image in subject.iterdir():
            split = image.name.split("_")

            ProcessFile(category.name, image, subjectID, subjectCounter)

            subjectCounter += 1
            currentFile +=1
        if subjectCounter > 0:
            subjectID += 1

## Train
data_loader arguments: `dataset` `model` `learning_rate` `epochs`
- Datasets: `oumvlp`, `casiab74`
- Models: `vgg16`, `resnet18`, `resnet50`
- Learning rate: positive float
- Epochs: positive int


### VGGNet 16

In [ ]:
!python data_loader.py "oumvlp" "vgg16" 0.00001 40

### ResNet 18

In [ ]:
!python data_loader.py "oumvlp" "resnet18" 0.0001 40

### ResNet 50

In [ ]:
!python data_loader.py "oumvlp" "resnet50" 0.0001 40